In [48]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

import catboost as ctb

In [2]:
data_dir = '/media/user01/wd1tb/wsdm_iqiyi'
train_dir = f'{data_dir}/train'
test_dir = f'{data_dir}/test'

In [3]:
# 测试集里只有 user_id 和 end_date

df_test = pd.read_csv(f'{test_dir}/test_without_label.csv')
print(df_test.shape)
df_test.head()

(50000, 2)


,user_id,end_date
0,10502904,204
1,10042083,208
2,10263103,178
3,10391836,204
4,10021774,191


In [4]:
# 启动日志, 只要出现在这个表里的都是当天留存的用户

df_launch = pd.read_csv(f'{train_dir}/app_launch_logs.csv')
print(df_launch.shape)
df_launch.head()

(8493878, 3)


,user_id,launch_type,date
0,10157996,0,129
1,10139583,0,129
2,10277501,0,129
3,10099847,0,129
4,10532773,0,129


In [5]:
# 先不用 launch_type, 后面再优化

df_launch.drop(['launch_type'], axis=1, inplace=True)
df_launch = df_launch.drop_duplicates(['user_id', 'date']).reset_index(drop=True)
print(df_launch.shape)
df_launch.head()

(8440083, 2)


,user_id,date
0,10157996,129
1,10139583,129
2,10277501,129
3,10099847,129
4,10532773,129


In [6]:
# 用户基础信息

df_user_portrait = pd.read_csv(f'{train_dir}/user_portrait_data.csv')
print(df_user_portrait.shape)
df_user_portrait.head()

(596906, 9)


,user_id,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code
0,10209854,2.0,5731,109581,1.0,2.0,0.0,1.0,865101.0
1,10230057,2.0,1877,20888,1.0,4.0,0.0,1.0,864102.0
2,10194990,2.0,7593,235438,2.0,3.0,1.0,1.0,866540.0
3,10046058,2.0,NaN,55137,1.0,4.0,0.0,1.0,NaN
4,10290885,2.0,2816,52431,1.0,4.0,0.0,0.0,NaN


In [7]:
# 用户交互表

df_user_interaction = pd.read_csv(f'{train_dir}/user_interaction_data.csv')
print(df_user_interaction.shape)
df_user_interaction.head()

(198608, 4)


,user_id,item_id,interact_type,date
0,10243056,22635954,1,213
1,10203565,24723827,3,213
2,10317559,20413036,2,213
3,10158940,23833050,1,213
4,10376271,22218154,1,213


In [8]:
# 用户观看时长表 (7100万条记录)

df_user_playback = pd.read_csv(f'{train_dir}/user_playback_data.csv')
print(df_user_playback.shape)
df_user_playback.head()

(71046035, 4)


,user_id,item_id,playtime,date
0,10057286,20628283.0,2208.612,145
1,10522615,23930557.0,31.054,145
2,10494028,20173699.0,115.952,145
3,10181987,21350426.0,1.585,145
4,10439175,22946929.0,51.726,145


In [9]:
# item 侧: 视频信息表

df_video = pd.read_csv(f'{train_dir}/video_related_data.csv')
print(df_video.shape)
df_video.head()

(4250105, 5)


,item_id,duration,father_id,tag_list,cast
0,24403453.0,6.0,NaN,50365080;50338575;50313222;50165986,NaN
1,22838795.0,7.0,NaN,50001708;50323515;50125414,NaN
2,24181187.0,5.0,NaN,50181045;50008354;50186586;50097060;50136213;5...,NaN
3,23549051.0,NaN,NaN,NaN,NaN
4,20276917.0,8.0,NaN,50286627;50257578;50415463,NaN


In [10]:
df_launch.date.max(), df_launch.date.min()

(222, 100)

In [11]:
df_launch = df_launch.sort_values(by=['user_id', 'date']).reset_index(drop=True)
df_user_date = df_launch.groupby(['user_id'])['date'].agg(list).reset_index()
df_user_date.head()

,user_id,date
0,10000000,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ..."
1,10000001,"[114, 117, 118]"
2,10000002,"[128, 129, 140, 144, 160, 163]"
3,10000003,[144]
4,10000004,"[154, 155, 156, 157]"


In [12]:
# 构建标签

labels = list()

for _, row in tqdm(df_user_date.iterrows()):
    date_list = row['date']
    
    # 循环到倒数第二个记录
    for idx, d in enumerate(date_list[:-1]):
        res = dict()
        res['user_id'] = row['user_id']
        days = 0
        # 从下一个记录开始算
        for da in date_list[idx+1:]:
            if da <= d + 7:
                days += 1
            else:
                break
        res['date'] = d
        res['label'] = days
        labels.append(res)

df_labels = pd.DataFrame(labels)
print(df_labels.shape)
df_labels.head()

0it [00:00, ?it/s]

(7840083, 3)


,user_id,date,label
0,10000000,131,1
1,10000000,132,0
2,10000000,141,0
3,10000000,164,0
4,10000000,179,1


In [13]:
# 如果 date 大于 222-7, 因为后续数据集并没有提供, 不知道是不是完整的数据, 先不使用

df_labels = df_labels[df_labels['date'] < 222 - 7]
print(df_labels.shape)

(7724778, 3)


In [14]:
df_labels.label.value_counts()

1    1285259
0    1184101
7    1171740
2     968604
3     822240
6     794019
4     756112
5     742703
Name: label, dtype: int64

In [15]:
# 合并用户特征

df_features = pd.merge(df_labels, df_user_portrait, on=['user_id'], how='left')
df_features.head()

,user_id,date,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code
0,10000000,131,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0
1,10000000,132,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0
2,10000000,141,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0
3,10000000,164,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0
4,10000000,179,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0


In [16]:
# 用户交互特征

df_user_interaction['user_date_count'] = df_user_interaction.groupby(
    ['user_id', 'date'])['user_id'].transform('count')

df_user_interaction['user_date_item_nunique'] = df_user_interaction.groupby(
    ['user_id', 'date'])['item_id'].transform('nunique')

df_user_interaction['user_date_interact_nunique'] = df_user_interaction.groupby(
    ['user_id', 'date'])['interact_type'].transform('nunique')

df_user_interaction = df_user_interaction.drop_duplicates(['user_id', 'date']).reset_index(drop=True)
df_user_interaction.drop(['item_id', 'interact_type'], axis=1, inplace=True)
df_user_interaction.head()

,user_id,date,user_date_count,user_date_item_nunique,user_date_interact_nunique
0,10243056,213,21,2,1
1,10203565,213,15,1,3
2,10317559,213,8,7,1
3,10158940,213,7,1,1
4,10376271,213,229,112,1


In [17]:
df_features = df_features.merge(df_user_interaction, on=['user_id', 'date'], how='left')
for col in ['user_date_count', 'user_date_item_nunique', 'user_date_interact_nunique']:
    df_features[col].fillna(0, inplace=True)
df_features.head()

,user_id,date,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,user_date_count,user_date_item_nunique,user_date_interact_nunique
0,10000000,131,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0
1,10000000,132,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0
2,10000000,141,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0
3,10000000,164,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0
4,10000000,179,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0


In [18]:
# 用户观看时长特征

df_user_playback['user_date_play_count'] = df_user_playback.groupby(
    ['user_id', 'date'])['user_id'].transform('count')

for method in ['sum', 'mean', 'max', 'min', 'std']:
    df_user_playback[f'user_date_playtime_{method}'] = df_user_playback.groupby(
        ['user_id', 'date'])['playtime'].transform(method)
    
df_user_playback = df_user_playback.drop_duplicates(['user_id', 'date']).reset_index(drop=True)
df_user_playback.drop(['item_id', 'playtime'], axis=1, inplace=True)
df_user_playback.head()

,user_id,date,user_date_play_count,user_date_playtime_sum,user_date_playtime_mean,user_date_playtime_max,user_date_playtime_min,user_date_playtime_std
0,10057286,145,3,3939.586,1313.195333,2208.612,110.134,1082.537025
1,10522615,145,97,3001.171,30.939907,954.561,0.407,99.206372
2,10494028,145,97,7321.931,75.483825,231.430,0.472,44.436529
3,10181987,145,213,5557.502,26.091559,480.868,0.006,60.431647
4,10439175,145,4,560.627,140.156750,243.930,24.674,118.255684


In [19]:
df_features = df_features.merge(df_user_playback, on=['user_id', 'date'], how='left')
for col in ['user_date_play_count', 'user_date_playtime_sum', 'user_date_playtime_mean',
            'user_date_playtime_max', 'user_date_playtime_min', 'user_date_playtime_std']:
    df_features[col].fillna(0, inplace=True)
df_features.head()

,user_id,date,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,user_date_count,user_date_item_nunique,user_date_interact_nunique,user_date_play_count,user_date_playtime_sum,user_date_playtime_mean,user_date_playtime_max,user_date_playtime_min,user_date_playtime_std
0,10000000,131,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0,12.0,22237.236,1853.103,2408.827,11.023,907.797062
1,10000000,132,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0,2.0,1249.204,624.602,1241.108,8.096,871.871146
2,10000000,141,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000000
3,10000000,164,0,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0,1.0,557.606,557.606,557.606,557.606,0.000000
4,10000000,179,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000000


In [20]:
# 构建测试集

df_test.columns = ['user_id', 'date']
df_test = pd.merge(df_test, df_user_portrait, on=['user_id'], how='left')
df_test = pd.merge(df_test, df_user_interaction, on=['user_id', 'date'], how='left')
df_test = pd.merge(df_test, df_user_playback, on=['user_id', 'date'], how='left')

print(df_test.shape)
df_test.head()

(50000, 19)


,user_id,date,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,user_date_count,user_date_item_nunique,user_date_interact_nunique,user_date_play_count,user_date_playtime_sum,user_date_playtime_mean,user_date_playtime_max,user_date_playtime_min,user_date_playtime_std
0,10502904,204,2.0,3588,53514,2.0,3.0,0.0,1.0,864115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10042083,208,2.0,3734,109449,2.0,4.0,0.0,1.0,864301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10263103,178,2.0,NaN,NaN,2.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10391836,204,2.0,5710,217160,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10021774,191,2.0,2805,52370,2.0,5.0,0.0,0.0,863401.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
for col in ['user_date_count', 'user_date_item_nunique',
    'user_date_interact_nunique', 'user_date_play_count',
    'user_date_playtime_sum', 'user_date_playtime_mean',
    'user_date_playtime_max', 'user_date_playtime_min',
    'user_date_playtime_std']:
    df_test[col].fillna(0, inplace=True)

In [22]:
df_test.head()

,user_id,date,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,user_date_count,user_date_item_nunique,user_date_interact_nunique,user_date_play_count,user_date_playtime_sum,user_date_playtime_mean,user_date_playtime_max,user_date_playtime_min,user_date_playtime_std
0,10502904,204,2.0,3588,53514,2.0,3.0,0.0,1.0,864115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10042083,208,2.0,3734,109449,2.0,4.0,0.0,1.0,864301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10263103,178,2.0,NaN,NaN,2.0,3.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10391836,204,2.0,5710,217160,1.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10021774,191,2.0,2805,52370,2.0,5.0,0.0,0.0,863401.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# 处理脏数据

def clean_data(x):
    if x == '':
        return -1
    else:
        return int(str(x).split(';')[0])
    
df_features['device_ram'].fillna('', inplace=True)
df_features['device_rom'].fillna('', inplace=True)
df_features['device_ram'] = df_features['device_ram'].apply(lambda x: clean_data(x))
df_features['device_rom'] = df_features['device_rom'].apply(lambda x: clean_data(x))

df_test['device_ram'].fillna('', inplace=True)
df_test['device_rom'].fillna('', inplace=True)
df_test['device_ram'] = df_test['device_ram'].apply(lambda x: clean_data(x))
df_test['device_rom'] = df_test['device_rom'].apply(lambda x: clean_data(x))

In [41]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7724782 entries, 0 to 7724781
Data columns (total 20 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   user_id                     int64  
 1   date                        int64  
 2   label                       int64  
 3   device_type                 float64
 4   device_ram                  int64  
 5   device_rom                  int64  
 6   sex                         float64
 7   age                         float64
 8   education                   float64
 9   occupation_status           float64
 10  territory_code              float64
 11  user_date_count             float64
 12  user_date_item_nunique      float64
 13  user_date_interact_nunique  float64
 14  user_date_play_count        float64
 15  user_date_playtime_sum      float64
 16  user_date_playtime_mean     float64
 17  user_date_playtime_max      float64
 18  user_date_playtime_min      float64
 19  user_date_playtime_st

In [43]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     50000 non-null  int64  
 1   date                        50000 non-null  int64  
 2   device_type                 49737 non-null  float64
 3   device_ram                  50000 non-null  int64  
 4   device_rom                  50000 non-null  int64  
 5   sex                         49358 non-null  float64
 6   age                         49230 non-null  float64
 7   education                   49099 non-null  float64
 8   occupation_status           49236 non-null  float64
 9   territory_code              47672 non-null  float64
 10  user_date_count             50000 non-null  float64
 11  user_date_item_nunique      50000 non-null  float64
 12  user_date_interact_nunique  50000 non-null  float64
 13  user_date_play_count        500

In [51]:
# 5 Folds traing

ycol = 'label' 
feature_names = list(
    filter(lambda x: x not in [ycol, 'date'], df_features.columns))

oof = []
prediction = df_test[['user_id', 'date']]
prediction[ycol] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_features[feature_names], df_features[ycol])):
    X_train = df_features.iloc[trn_idx][feature_names]
    Y_train = df_features.iloc[trn_idx][ycol]

    X_val = df_features.iloc[val_idx][feature_names]
    Y_val = df_features.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))
    
    params = {
        'task_type': 'GPU',
        'learning_rate': 0.1,
        'eval_metric': 'RMSE',
        'loss_function': 'RMSE',
        'iterations': 10000,
        'random_seed': 2021,
        'max_depth': 8,
        'reg_lambda': 0.5,
        'early_stopping_rounds': 100,
        'use_best_model': True
    }
    
    model = ctb.CatBoostRegressor(**params)

    model.fit(X_train,
              Y_train,
              eval_set=(X_val, Y_val),
              verbose=100)

    pred_val = model.predict(X_val)
    df_oof = df_features.iloc[val_idx][['user_id', 'date', ycol]].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = model.predict(df_test[feature_names])
    prediction[ycol] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

0:	learn: 2.3783161	test: 2.3783343	best: 2.3783343 (0)	total: 33.9ms	remaining: 5m 39s
100:	learn: 2.0875441	test: 2.0876263	best: 2.0876263 (100)	total: 2.66s	remaining: 4m 20s
200:	learn: 2.0853977	test: 2.0857227	best: 2.0857219 (180)	total: 5.2s	remaining: 4m 13s
300:	learn: 2.0853885	test: 2.0857125	best: 2.0857125 (300)	total: 7.8s	remaining: 4m 11s
400:	learn: 2.0853789	test: 2.0857065	best: 2.0857063 (398)	total: 10.4s	remaining: 4m 8s
500:	learn: 2.0853701	test: 2.0856990	best: 2.0856990 (500)	total: 12.9s	remaining: 4m 5s
600:	learn: 2.0853628	test: 2.0856942	best: 2.0856941 (586)	total: 15.5s	remaining: 4m 2s
700:	learn: 2.0853575	test: 2.0856892	best: 2.0856891 (698)	total: 18.1s	remaining: 3m 59s
800:	learn: 2.0853533	test: 2.0856842	best: 2.0856842 (797)	total: 20.7s	remaining: 3m 57s
900:	learn: 2.0853466	test: 2.0856793	best: 2.0856781 (896)	total: 23.2s	remaining: 3m 54s
1000:	learn: 2.0853398	test: 2.0856734	best: 2.

3400:	learn: 2.0831462	test: 2.0842343	best: 2.0842336 (3394)	total: 1m 28s	remaining: 2m 52s
3500:	learn: 2.0831037	test: 2.0841967	best: 2.0841967 (3500)	total: 1m 31s	remaining: 2m 49s
3600:	learn: 2.0830819	test: 2.0841807	best: 2.0841804 (3599)	total: 1m 34s	remaining: 2m 47s
3700:	learn: 2.0830143	test: 2.0841231	best: 2.0841222 (3686)	total: 1m 36s	remaining: 2m 44s
3800:	learn: 2.0829885	test: 2.0841038	best: 2.0841032 (3795)	total: 1m 39s	remaining: 2m 41s
3900:	learn: 2.0829684	test: 2.0840836	best: 2.0840836 (3900)	total: 1m 41s	remaining: 2m 39s
4000:	learn: 2.0829385	test: 2.0840597	best: 2.0840590 (3999)	total: 1m 44s	remaining: 2m 36s
4100:	learn: 2.0829077	test: 2.0840311	best: 2.0840311 (4100)	total: 1m 47s	remaining: 2m 34s
4200:	learn: 2.0828959	test: 2.0840204	best: 2.0840200 (4187)	total: 1m 49s	remaining: 2m 31s
4300:	learn: 2.0828907	test: 2.0840141	best: 2.0840136 (4281)	total: 1m 52s	remaining: 2m 28s
4400:	learn: 2.0828861	test: 2.0840101	best: 2.0840096 (4391

2000:	learn: 2.0851980	test: 2.0852820	best: 2.0852814 (1997)	total: 52.8s	remaining: 3m 30s
2100:	learn: 2.0851932	test: 2.0852778	best: 2.0852778 (2099)	total: 55.4s	remaining: 3m 28s
2200:	learn: 2.0851904	test: 2.0852772	best: 2.0852766 (2178)	total: 58s	remaining: 3m 25s
bestTest = 2.085276608
bestIteration = 2178
Shrink model to first 2179 iterations.

Fold_5 Training ================================

0:	learn: 2.3775910	test: 2.3776607	best: 2.3776607 (0)	total: 34ms	remaining: 5m 39s
100:	learn: 2.0871557	test: 2.0880413	best: 2.0880413 (100)	total: 2.63s	remaining: 4m 17s
200:	learn: 2.0848333	test: 2.0859599	best: 2.0859599 (200)	total: 5.19s	remaining: 4m 13s
300:	learn: 2.0843108	test: 2.0854550	best: 2.0854550 (300)	total: 7.8s	remaining: 4m 11s
400:	learn: 2.0842692	test: 2.0854177	best: 2.0854167 (378)	total: 10.4s	remaining: 4m 9s
500:	learn: 2.0842647	test: 2.0854125	best: 2.0854125 (493)	total: 13.1s	remaining: 4m 8s
600:	learn: 2.0842620	test: 2.0854099	best: 2.08540

In [90]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg( 'mean').sort_values(ascending=False).reset_index() 
df_importance

,column,importance
0,user_date_playtime_sum,29.312528
1,user_date_playtime_max,9.101296
2,device_type,8.003358
3,education,7.891915
4,user_date_play_count,7.745806
5,occupation_status,6.849918
6,territory_code,6.022400
7,age,4.257183
8,device_ram,4.093966
9,user_date_playtime_std,4.023236


In [54]:
df_oof = pd.concat(oof)
df_oof.head()

,user_id,date,label,pred
8,10000000,191,6,1.787953
12,10000000,196,3,1.787953
18,10000001,117,1,4.376158
24,10000004,154,3,3.841857
25,10000004,155,2,3.272177


In [61]:
print('线下指标得分:', 1 - mean_absolute_error(df_oof['label'], df_oof['pred']) / 7)

线下指标得分: 0.7495230836495929


In [76]:
prediction.head()

,user_id,date,label
0,10502904,204,1.807573
1,10042083,208,1.762614
2,10263103,178,0.720667
3,10391836,204,1.318699
4,10021774,191,1.334374


In [81]:
ss = pd.read_csv(f'{data_dir}/sample-a.csv', header=None)
ss.columns = ['user_id', 'label']
ss.head()

,user_id,label
0,10007813,3.26
1,10052988,6.02
2,10279068,3.22
3,10546696,4.70
4,10406659,3.34


In [83]:
ss = pd.merge(ss[['user_id']], prediction[['user_id', 'label']], on='user_id', how='left')
ss.head()

,user_id,label
0,10007813,1.324751
1,10052988,1.172367
2,10279068,1.704240
3,10546696,1.385710
4,10406659,1.973898


In [86]:
ss.to_csv('baseline.csv', header=None, index=False, float_format='%.2f')

In [87]:
!tail baseline.csv

10471862,1.39
10272071,2.85
10055491,2.22
10065542,4.30
10039248,2.45
10355586,1.49
10589773,1.73
10181954,0.87
10544736,2.72
10354569,0.81
